Source:
- https://github.com/abhinavsagar/breast-cancer-classification/blob/master/Breast_cancer_classification.ipynb

## Libraries

In [2]:
import numpy as np
import json
import utils_nn_eval
import random
import gc
import pandas
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras import backend as K
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import cohen_kappa_score, accuracy_score

In [3]:
import json
import math
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import tensorflow as tf
import gc
import itertools

from PIL import Image
from functools import partial
from sklearn import metrics
from collections import Counter
from keras import backend as K
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score

from keras import layers
from tensorflow.keras.applications import DenseNet201
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [15]:
figure_path = 'D:/FH/Kurse/semester_3/medical_imaging/Medical_Imaging_Elias/Figures/'

## Load Images

## Gan Images

In [4]:
benign_gan = utils_nn_eval.get_all_images('gan_benign')
malignant_gan = utils_nn_eval.get_all_images('gan_malignant')

In [5]:
print(len(benign_gan))
print(len(malignant_gan))

300
300


### Ground Truth Images

In [6]:
benign_gt = utils_nn_eval.get_all_images('gt_benign')
malignant_gt = utils_nn_eval.get_all_images('gt_malignant')

In [7]:
print(len(benign_gt))
print(len(malignant_gt))

223
301


In [8]:
random.Random(4).shuffle(benign_gan)
random.Random(4).shuffle(malignant_gan)
random.Random(4).shuffle(benign_gt)
random.Random(4).shuffle(malignant_gt)

In [9]:
benign_temp_gan = benign_gan[0:150]
malignant_temp_gan = malignant_gan[0:150]
benign_temp_gt = benign_gt[0:150]
malignant_temp_gt = malignant_gt[0:150]

In [10]:
benign = np.concatenate((benign_temp_gan, benign_temp_gt))
malignant = np.concatenate((malignant_temp_gan, malignant_temp_gt))

In [11]:
random.Random(4).shuffle(benign)
random.Random(4).shuffle(malignant)

## Create Split

In [12]:
X_test, Y_test, x_train, x_val, y_train, y_val = utils_nn_eval.create_split(benign, malignant)

## Data Generator

In [13]:
train_generator = utils_nn_eval.create_train_generator()

## Model Creation

In [14]:
def build_model(backbone, lr = 1e-4):

    model = Sequential()
    model.add(backbone)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(2, activation = 'softmax'))
    
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = Adam(learning_rate = lr),
        metrics = ['accuracy']
    )
    
    return model

In [15]:
K.clear_session()
gc.collect()

shape = (128, 128, 3)

resnet = DenseNet201(
                        weights = 'imagenet',
                        include_top = False,
                        input_shape = shape
                    )

model = build_model(resnet , lr = 1e-4)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 4, 4, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1920)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1920)              7680      
_________________________________________________________________
dense (Dense)                (None, 2)                 3842      
Total params: 18,333,506
Trainable params: 18,100,610
Non-trainable params: 232,896
_________________________________________________________________


## Training & Evaluation

In [16]:
# Learning Rate Reducer
learn_control = ReduceLROnPlateau(monitor='val_accuracy', patience=5,
                                  verbose=1,factor=0.2, min_lr=1e-7)

# Checkpoint
filepath = "densenet_mixed_local.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [17]:
history = model.fit(

    train_generator.flow(x_train, y_train, batch_size = 16),
    steps_per_epoch=x_train.shape[0] / 16,
    epochs=50,
    validation_data=(x_val, y_val),
    callbacks=[learn_control, checkpoint]
)

Epoch 1/50
24/24 [==============================] - 51s 1s/step - loss: 0.6285 - accuracy: 0.6693 - val_loss: 0.7769 - val_accuracy: 0.5208

Epoch 00001: val_accuracy improved from -inf to 0.52083, saving model to densenet_mixed_local.hdf5
Epoch 2/50
24/24 [==============================] - 30s 1s/step - loss: 0.3605 - accuracy: 0.8542 - val_loss: 0.5842 - val_accuracy: 0.8125

Epoch 00002: val_accuracy improved from 0.52083 to 0.81250, saving model to densenet_mixed_local.hdf5
Epoch 3/50
24/24 [==============================] - 32s 1s/step - loss: 0.2923 - accuracy: 0.8958 - val_loss: 0.7132 - val_accuracy: 0.5000

Epoch 00003: val_accuracy did not improve from 0.81250
Epoch 4/50
24/24 [==============================] - 31s 1s/step - loss: 0.2407 - accuracy: 0.9219 - val_loss: 0.8401 - val_accuracy: 0.4792

Epoch 00004: val_accuracy did not improve from 0.81250
Epoch 5/50
24/24 [==============================] - 32s 1s/step - loss: 0.1670 - accuracy: 0.9323 - val_loss: 1.1107 - val_ac

In [25]:
with open(r'D:\FH\Kurse\semester_3\medical_imaging\repos\Medical_Imaging_Elias\ResNet_DenseNet_Model\Densenet_New\history_mixed.json', 'w') as f:
    
    json.dump(str(history.history), f)

In [20]:
model.save('./densenet_mixed-data')

INFO:tensorflow:Assets written to: ./densenet_mixed-data\assets


## Prediction on Ground Truth Data

In [30]:
filepath_model = "./densenet_mixed_local.hdf5"
model.load_weights(filepath_model)

In [18]:
Y_val_pred = model.predict(x_val)

In [19]:
accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1))

0.96875

In [22]:
Y_test_pred = model.predict(X_test)

In [23]:
accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_test_pred, axis=1))

0.925